In [1]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import open3d as o3d
import time
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
img = cv2.imread("../dataset/rgbd/rgbd_depth/depth_009_05.png")
img_pil = Image.open("../dataset/rgbd/rgbd_depth/depth_009_05.png")
# img_np = np.load("../dataset/rgbd/rgbd_depth/depth_002_03.png")

In [3]:
ranges_x = np.arange(0,255,0.1)
pil_data = np.asarray(list(img_pil.getdata())).reshape((img_pil.width,img_pil.height))
print(pil_data[101,101] * 0.000124987)


0.540193814


In [13]:
# img = Image.open("C:/Users/david/Pictures/realsense/depth_02_Depth.png").convert("L")
# img = o3d.io.read_image("C:/Users/david/Pictures/06_Depth.png")
# rgb_image = o3d.io.read_image("C:/Users/david/Pictures/06_Color.png")


img = o3d.io.read_image      ("C:/Users/david/Pictures/realsense/test_set_full/day_05/depth/depth_009.png")
rgb_image = o3d.io.read_image("C:/Users/david/Pictures/realsense/test_set_full/day_05/color/color_009.png")


# img = np.asarray(img)
# print(img.shape)
# depth_img = o3d.geometry.Image(img)
# intrinsics = np.array([[475.903, 0, 315.53],[0, 475.903, 245.44],[0, 0, 1]])


img_o3d = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb_image, img, depth_scale=1000)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    img_o3d,
    o3d.camera.PinholeCameraIntrinsic(
        1280, 720, 637.735, 637.735, 645.414, 349.205))
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

o3d.visualization.draw_geometries([pcd])



In [5]:
area = np.load("dati1.txt.npy")

pcd = o3d.geometry.PointCloud()

pcd.points = o3d.utility.Vector3dVector(area)

o3d.geometry.PointCloud.estimate_normals(pcd, search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=1, max_nn=1))
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 10 * avg_dist

mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
           pcd,
           o3d.utility.DoubleVector([radius, radius * 2]))

mesh.paint_uniform_color([0.9, 0.1, 0.1])


o3d.visualization.draw_geometries([mesh])

print(mesh.get_surface_area())



0.0014975663670125164


In [42]:
# im_d = np.array(Image.open("C:/Users/david/Pictures/06_Depth.png").convert("L"))
# im_d = cv2.imread("./test_depth.png", 0)
imported = Image.open("../test_depth.png")
im_d = np.array(list(imported.getdata())).reshape((imported.height,imported.width))
intrinsics = np.array([[606.018, 0, 318.183], [0, 604.975, 246.266], [0, 0, 1]])


z = im_d * 0.001
position = 100
x = (position - intrinsics[0,2]) / intrinsics[0,0] * z[position,position]
y = (position - intrinsics[1,2]) / intrinsics[1,1] * z[position,position]
print(z[position,position])
print(x, y)

0.99
-0.35642698731720834 -0.23935425430802923


In [7]:
# import pyvista as pv
# # 
# img = Image.open("C:/Users/david/Pictures/realsense/depth_02_Depth.png")
# cloud = pv.PolyData(img)
# surf = cloud.delaunay_2d()
# surf.plot(show_edges=True)
# print(surf.area)

In [2]:
with open("metrics.json", 'r') as f:
        leaf_data = json.load(f)

In [3]:
# print(leaf_data["leafs"][-1]["xyz_position"][0])
print(leaf_data["leafs"][-1]["image_name"])
data = np.array(leaf_data["leafs"][-1]["xyz_position"]).reshape((-1,3))

print(data.shape)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data)

o3d.visualization.draw_geometries([pcd])

color_009.png
(7340, 3)
